In [1]:
pip install --upgrade scikit-learn==1.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 60.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.13.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.0.2 which is incompatible.


In [2]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import torch.nn as nn
import torch
import joblib
import os

from google.colab import drive
drive.mount('/content/drive')

!pip install scikit-tensor-py3==0.2.1
from sktensor import dtensor

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 1.4 MB/s eta 0:00:00


In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA")
else:
    device = torch.device('cpu')

CUDA


In [7]:
# Load Min-max scaler models

scaler_dic = dict()
features = ['calories', 'distance']

dir = '/content/drive/MyDrive/P3 Fitrec Dataset/scaler/'

for feature in features:
    path = os.path.join(dir, 'scaler_'+feature+'_2.m')
    scaler_dic[feature] = joblib.load(path)
    scaler_dic[feature].clip = False

# Load onehot encoder for categorical features

dir = '/content/drive/MyDrive/P3 Fitrec Dataset/scaler/OneHotEncoder.m'

OneHot_enc = joblib.load(dir)

# Load user embedding and route embedding

user_dir = '/content/drive/MyDrive/P3 Fitrec Dataset/Generated datsets fresh/EMBEDDINGS/user_embedding_9.m'
route_dir = '/content/drive/MyDrive/P3 Fitrec Dataset/Generated datsets fresh/EMBEDDINGS/route_embedding_9.m'

user_embed_df = joblib.load(user_dir)
route_embed_df = joblib.load(route_dir)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import torch.nn as nn
import torch

# Define model

import torch.nn as nn

class DisReg_MLP_2Layer(nn.Module):
    def __init__(self, input_dim, hidden_dim_1, p):
        super(DisReg_MLP_2Layer, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim_1)
        self.fc2 = nn.Linear(hidden_dim_1, 1)

        self.act_1 = nn.ReLU()
        self.act_2 = nn.Sigmoid()

        self.drop_1 = nn.Dropout(p)

    def forward(self, data):

        output = self.fc1(data)
        output = self.act_1(output)
        output = self.drop_1(output)

        output = self.fc2(output)
        output = self.act_2(output)

        return output


# Define function to load model

def load_model(filename, model):

    DATA_PATH = "/content/drive/MyDrive/P3 Fitrec Dataset/Generated datsets fresh/Distance_Model/"+filename

    checkpoint = torch.load(DATA_PATH)
    model.load_state_dict(checkpoint['best_model_state_dict'])

    return model

# Load Model

LOAD_MODEL_NAME = 'DisReg_MLP_2L_3'
INPUT_DIM = 14
HIDDEN_DIM = 64
DROP_OUT = 0.2

# Create an instance of your model
model = DisReg_MLP_2Layer(INPUT_DIM, HIDDEN_DIM, DROP_OUT).to(device)

# Load the state dictionary into the model instance
model = load_model(LOAD_MODEL_NAME, model)


In [ ]:
cols = ['id', 'userId', 'gender', 'sport',
        'calories', 'route_id', 'distance_adjusted_sum']

In [ ]:
# Take User Inputs

USER_ID = 3743021
GENDER = 'male'
SPORT = 'run'
CALORIES = 1035
WORKOUT_ID = 216633190

In [ ]:
test_df=pd.read_csv('/content/drive/MyDrive/ML_DataSets/P3 Fitrec Dataset/Generated datsets fresh/TestData_adjusted.csv')

In [ ]:
len(test_df)

13264

In [ ]:
test_df.head(10)

,id,userId,gender,sport,duration,calories,distance,avg_heart_rate,longitude,latitude,...,Cluster,route,route_id,isback,altitude_adjusted,distance_adjusted,complete_idx,speed_adjusted,heart_rate_adjusted,distance_adjusted_sum
0,44714694,3029335,male,bike,3093,673.000,28.438030,158.938,"[12.30067047290504, 12.30064365081489, 12.3007...","[55.75307199731469, 55.75303830206394, 55.7531...",...,3,"('bike', '3')",7,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.6, -1.0...","[0.004106793894803813, 0.013695662971930606, 0...",497,"[7.392229010646863, 8.217397783158363, 11.1197...","[101, 100, 108, 116, 119, 121, 125, 127, 129, ...",28.438030
1,216633190,3743021,male,run,5038,1035.000,20.688540,168.218,"[-3.956668172031641, -3.9564789086580276, -3.9...","[40.68045811727643, 40.68047320470214, 40.6804...",...,2,"('run', '2')",5,0,"[980.4, 980.4, 980.4, 980.4, 982.0, 983.6, 984...","[0.016052681710459784, 0.02339216015239165, 0....",490,"[4.815804513137935, 8.446735463895832, 9.86672...","[115, 123, 130, 140, 152, 155, 155, 158, 165, ...",20.688540
2,39917558,3714939,male,bike,4713,1096.000,29.506422,116.630,"[175.760039, 175.760115, 175.760119, 175.76025...","[-40.290315, -40.290278, -40.290225, -40.28971...",...,0,"('bike', '0')",6,0,"[77.784, 77.631, 77.563, 77.597, 77.577, 78.11...","[0.007651078525645552, 0.005905333200176488, 0...",491,"[6.885970673080997, 10.629599760317678, 16.010...","[72, 71, 71, 80, 85, 87, 90, 96, 96, 106, 107,...",29.506422
3,565437308,6307695,male,bike,3485,1096.000,20.313390,140.512,"[-97.03839867375791, -97.03846111893654, -97.0...","[33.04064305499196, 33.04065956734121, 33.0407...",...,3,"('bike', '3')",7,0,"[191.4, 192.0, 192.6, 193.8, 194.2, 194.6, 194...","[0.00613474822206398, 0.015518684566665284, 0....",494,"[4.417018719886065, 9.31121073999917, 14.80457...","[117, 116, 112, 115, 116, 116, 113, 114, 117, ...",20.313390
4,475961251,519009,male,run,5537,1507.000,20.994730,167.948,"[-0.519849993288517, -0.5196931678801775, -0.5...","[38.98970796726644, 38.989733615890145, 38.989...",...,1,"('run', '1')",3,0,"[299.8, 297.8, 288.8, 281.6, 268.8, 265.0, 245...","[0.013998747348355028, 0.026071180334315595, 0...",492,"[8.399248409013016, 7.679626506429677, 9.64103...","[145, 153, 154, 159, 166, 160, 165, 169, 167, ...",20.994730
5,460789358,13577416,male,bike,1554,335.772,9.434300,142.748,"[-6.274891, -6.274938, -6.274938, -6.274955, -...","[53.331026, 53.331063, 53.331074, 53.3311, 53....",...,3,"('bike', '3')",7,0,"[0.0, 0.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, ...","[0.005165677240793355, 0.001239766579462859, 0...",493,"[6.198812688952026, 1.4877198953554307, 3.7255...","[88, 87, 87, 89, 93, 98, 104, 108, 110, 109, 1...",9.434300
6,413541825,3537163,male,bike,2625,358.685,14.700000,138.418,"[6.14922, 6.149543, 6.149894, 6.15012, 6.15044...","[46.1964, 46.196409, 46.196245, 46.196114, 46....",...,1,"('bike', '1')",8,0,"[396.6, 396.2, 398.0, 399.0, 400.0, 400.0, 400...","[0.024891773219604396, 0.03265437960568323, 0....",492,"[17.922076718115164, 23.511153316091924, 16.35...","[115, 116, 120, 125, 130, 134, 137, 138, 138, ...",14.700000
7,205708147,8994885,male,run,3241,724.000,10.000000,152.668,"[30.509890168905258, 30.50995278172195, 30.509...","[50.52573949098587, 50.52572431974113, 50.5256...",...,1,"('run', '1')",3,0,"[-61.4, -58.8, -55.0, -46.8, -36.2, -34.4, -18...","[0.005404662305942694, 0.0063818203375429405, ...",492,"[4.864196075348425, 7.125504168232097, 7.97617...","[109, 114, 119, 124, 128, 129, 134, 134, 135, ...",10.000000
8,433622196,797841,male,mountain bike,1646,218.000,9.658580,121.592,"[-5.310257105156779, -5.310270935297012, -5.31...","[50.213147811591625, 50.21314018405974, 50.213...",...,1,"('mountain bike', '1')",11,0,"[86.0, 86.0, 86.0, 86.0, 86.0, 86.0, 86.0, 86....","[0.0012995732923114605, 0.015340037959431322, ...",492,"[4.678463852321258, 7.889162379136108, 7.28458...","[88, 88, 90, 94, 98, 99, 98, 98, 100, 100, 110...",9.658580
9,637314737,478585,male,run,5525,1128.190,16.460

In [ ]:
print(test_df.columns)

Index(['id', 'userId', 'gender', 'sport', 'duration', 'calories', 'distance',
       'avg_heart_rate', 'longitude', 'latitude', 'altitude', 'timestamp',
       'heart_rate', 'speed', 'url', 'derived_distance', 'derived_speed',
       'time_elapsed', 'validate', 'avg_alti', 'change_alti', 'max_alti',
       'min_alti', 'diff_alti', 'avg_speed', 'Cluster', 'route', 'route_id',
       'isback', 'altitude_adjusted', 'distance_adjusted', 'complete_idx',
       'speed_adjusted', 'heart_rate_adjusted', 'distance_adjusted_sum'],
      dtype='object')


In [ ]:
# find Route_id based on workout id
Route_id = test_df.loc[test_df.id == WORKOUT_ID, 'route_id'].to_numpy()[0]
# find route total distance based on workout id
distance_adjusted_sum = test_df.loc[test_df.id ==
                                    WORKOUT_ID, 'distance_adjusted_sum'].to_numpy()[0]
# find userEmbedding based on userId
user_embed = np.array(
    user_embed_df[user_embed_df.userId == USER_ID].userEmbed.values[0])
# find routeEmbedding based on routeId
route_embed = np.array(
    route_embed_df[route_embed_df.route_id == Route_id].routeEmbed.values[0])

In [ ]:
def data_process(gender, sport, calories, Route_id, distance_adjusted_sum, user_embed, route_embed):
    # label encode gender

    choices = [0, 1, 2]
    conditions = [
        (gender == 'male'),
        (gender == 'female'),
        (gender == 'unknown')]

    genderId = np.select(conditions, choices, default=0)

    # label encode sport

    conditions = [
        (sport == 'run'),
        (sport == 'bike'),
        (sport == 'mountain bike')]

    sportId = np.select(conditions, choices, default=0)

    # scale calories
    calories_scaled = scaler_dic['calories'].transform(
        np.array(calories).reshape(-1, 1))[0]
    # scale distance
    total_distance_scaled = scaler_dic['distance'].transform(
        np.array(distance_adjusted_sum).reshape(-1, 1))[0]
    # one hot encode genderId and sportId
    gender_sport_onehot = OneHot_enc.transform(
        np.hstack((genderId, sportId)).reshape(1, -1)).toarray()[0]
    # concatenate input features into numpy array
    data_input = np.hstack((calories_scaled, total_distance_scaled,
                            user_embed, route_embed, gender_sport_onehot))

    return data_input


data_input = data_process(
    GENDER,
    SPORT,
    CALORIES,
    Route_id,
    distance_adjusted_sum,
    user_embed,
    route_embed)

In [ ]:
model.eval()
with torch.no_grad():
    pred = model(torch.Tensor(data_input).to(device)).item()

# convert predicted distance to km
pred_km = scaler_dic['distance'].inverse_transform(
    np.array(pred).reshape(-1, 1))[0][0]

In [ ]:
print('Predicted distance: {:.4} km'.format(pred_km))
print('Workout route total distance: {:.4} km'.format(distance_adjusted_sum))

Predicted distance: 19.28 km
Workout route total distance: 20.69 km


In [ ]:
import pandas as pd

# Assuming you have your test data in a DataFrame named test_df
# Replace this DataFrame with your actual test data
# test_df = pd.read_csv('test_data.csv')  # Load your test data from a CSV file or another source

# Create lists to store data
id_list = []  # To store the 'id' column
true_values_list = []
predicted_values_list = []
gender_list = []
sport_list = []
calories_list = []
Route_id_list = []
distance_adjusted_sum_list = []
user_id_list = []

for index, row in test_df.iterrows():
    id = row['id']  # Extract 'id' from the 'test_df'
    gender = row['gender']
    sport = row['sport']
    calories = row['calories']
    Route_id = row['route_id']
    distance_adjusted_sum = row['distance_adjusted_sum']
    user_id = row['userId']

    # Fetch userEmbedding and routeEmbedding from your respective data sources
    user_embed = np.array(user_embed_df[user_embed_df.userId == user_id].userEmbed.values[0])
    route_embed = np.array(route_embed_df[route_embed_df.route_id == Route_id].routeEmbed.values[0])

    # Rest of your data processing code remains the same
    data_input = data_process(gender, sport, calories, Route_id, distance_adjusted_sum, user_embed, route_embed)

    # Make predictions
    model.eval()
    with torch.no_grad():
        pred = model(torch.Tensor(data_input).to(device)).item()

    # Convert predicted distance to km
    pred_km = scaler_dic['distance'].inverse_transform(np.array(pred).reshape(-1, 1))[0][0]

    # Append true and predicted values and the other attributes to the respective lists
    id_list.append(id)
    true_values_list.append(distance_adjusted_sum)
    predicted_values_list.append(pred_km)
    gender_list.append(gender)
    sport_list.append(sport)
    calories_list.append(calories)
    Route_id_list.append(Route_id)
    distance_adjusted_sum_list.append(distance_adjusted_sum)
    user_id_list.append(user_id)

# Create a DataFrame to store or further analyze the true and predicted values and other attributes
results_df = pd.DataFrame({
    'id': id_list,  # Include 'id' in the results_df
    'True_Distance': true_values_list,
    'Predicted_Distance': predicted_values_list,
    'Gender': gender_list,
    'Sport': sport_list,
    'Calories': calories_list,
    'Route_id': Route_id_list,
    'Distance_Adjusted_Sum': distance_adjusted_sum_list,
    'User_id': user_id_list
})

# You can also save the results to a CSV file if needed
results_df.to_csv('prediction_results.csv', index=False)

In [ ]:
results_df

,id,True_Distance,Predicted_Distance,Gender,Sport,Calories,Route_id,Distance_Adjusted_Sum,User_id
0,44714694,28.438030,26.825425,male,bike,673.00,7,28.438030,3029335
1,216633190,20.688540,19.283408,male,run,1035.00,5,20.688540,3743021
2,39917558,29.506422,28.533621,male,bike,1096.00,6,29.506422,3714939
3,565437308,20.313390,19.866239,male,bike,1096.00,7,20.313390,6307695
4,475961251,20.994730,20.227541,male,run,1507.00,3,20.994730,519009
...,...,...,...,...,...,...,...,...,...
13259,350645350,12.198151,11.976955,male,run,1091.00,3,12.198151,3702346
13260,366920375,65.961828,59.986019,male,bike,1219.00,8,65.961828,12645302
13261,477619182,7.297215,6.579377,male,run,370.00,4,7.297215,1655221
13262,315951583,24.159650,22.307479,male,run,1680.24,4,24.159650,9336459


#2 MLP size 3 embedding

**RMSE**

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming you have a DataFrame named results_df with 'True_Distance' and 'Predicted_Distance' columns

true_distances = results_df['True_Distance']
predicted_distances = results_df['Predicted_Distance']

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_distances, predicted_distances))

print("Root Mean Square Error (RMSE) for the entire dataset:", rmse)


Root Mean Square Error (RMSE) for the entire dataset: 1.77355777176449


In [ ]:
import pandas as pd

# Assuming you have your test data in a DataFrame named test_df
# Replace this DataFrame with your actual test data
# test_df = pd.read_csv('test_data.csv')  # Load your test data from a CSV file or another source

# Create lists to store data
id_list = []  # To store the 'id' column
true_values_list = []
predicted_values_list = []
gender_list = []
sport_list = []
calories_list = []
Route_id_list = []
distance_adjusted_sum_list = []
user_id_list = []

# Process the first half of the rows in test_df
for index, row in test_df.head(len(test_df)//2).iterrows():
    id = row['id']  # Extract 'id' from the 'test_df'
    gender = row['gender']
    sport = row['sport']
    calories = row['calories']
    Route_id = row['route_id']
    distance_adjusted_sum = row['distance_adjusted_sum']
    user_id = row['userId']

    # Fetch userEmbedding and routeEmbedding from your respective data sources
    user_embed = np.array(user_embed_df[user_embed_df.userId == user_id].userEmbed.values[0])
    route_embed = np.array(route_embed_df[route_embed_df.route_id == Route_id].routeEmbed.values[0])

    # Rest of your data processing code remains the same
    data_input = data_process(gender, sport, calories, Route_id, distance_adjusted_sum, user_embed, route_embed)

    # Make predictions
    model.eval()
    with torch.no_grad():
        pred = model(torch.Tensor(data_input).to(device)).item()

    # Convert predicted distance to km
    pred_km = scaler_dic['distance'].inverse_transform(np.array(pred).reshape(-1, 1))[0][0]

    # Append true and predicted values and the other attributes to the respective lists
    id_list.append(id)
    true_values_list.append(distance_adjusted_sum)
    predicted_values_list.append(pred_km)
    gender_list.append(gender)
    sport_list.append(sport)
    calories_list.append(calories)
    Route_id_list.append(Route_id)
    distance_adjusted_sum_list.append(distance_adjusted_sum)
    user_id_list.append(user_id)

# Create a DataFrame to store or further analyze the true and predicted values and other attributes
results_df2 = pd.DataFrame({
    'id': id_list,  # Include 'id' in the results_df
    'True_Distance': true_values_list,
    'Predicted_Distance': predicted_values_list,
    'Gender': gender_list,
    'Sport': sport_list,
    'Calories': calories_list,
    'Route_id': Route_id_list,
    'Distance_Adjusted_Sum': distance_adjusted_sum_list,
    'User_id': user_id_list
})

# You can also save the results to a CSV file if needed
results_df2.to_csv('prediction_results_half.csv', index=False)

# Display or further analyze the results as required


In [ ]:
results_df2

,id,True_Distance,Predicted_Distance,Gender,Sport,Calories,Route_id,Distance_Adjusted_Sum,User_id
0,44714694,28.438030,26.825425,male,bike,673.000,7,28.438030,3029335
1,216633190,20.688540,19.283408,male,run,1035.000,5,20.688540,3743021
2,39917558,29.506422,28.533621,male,bike,1096.000,6,29.506422,3714939
3,565437308,20.313390,19.866239,male,bike,1096.000,7,20.313390,6307695
4,475961251,20.994730,20.227541,male,run,1507.000,3,20.994730,519009
...,...,...,...,...,...,...,...,...,...
6627,179640463,15.010000,14.019518,male,run,853.000,1,15.010000,5979382
6628,442392659,10.010000,9.575350,male,run,637.000,1,10.010000,6946869
6629,651439934,5.328561,5.661406,male,run,475.195,1,5.328561,3558042
6630,434588699,14.563460,14.379180,male,bike,446.000,7,14.563460,6692748


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming you have a DataFrame named results_df with 'True_Distance' and 'Predicted_Distance' columns

true_distances = results_df2['True_Distance']
predicted_distances = results_df2['Predicted_Distance']

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_distances, predicted_distances))

print("Root Mean Square Error (RMSE) for the entire dataset:", rmse)

Root Mean Square Error (RMSE) for the entire dataset: 1.3444239051201268


**MAE**

In [ ]:
from sklearn.metrics import mean_absolute_error

# Assuming you have a DataFrame named results_df with 'True_Distance' and 'Predicted_Distance' columns

true_distances = results_df2['True_Distance']
predicted_distances = results_df2['Predicted_Distance']

# Calculate the MAE
mae = mean_absolute_error(true_distances, predicted_distances)

print("Mean Absolute Error (MAE) for the entire dataset:", mae)


Mean Absolute Error (MAE) for the entire dataset: 0.921188722933266
